# Principali modifiche alle funzioni in alpha_complex.jl

Vecchia versione della funzione alphaFilter

In [1]:
function alphaFilter(
        V::Lar.Points,
        DT = Array{Int64,1}[];
        digits=64
    )::DataStructures.SortedDict{}

    dim = size(V, 1)
    filtration = DataStructures.SortedDict{Array{Int64,1},Float64}()

    # 1 - Each point => alpha_char = 0.
    for i = 1 : size(V, 2)
        insert!(filtration, [i], 0.)
    end

    # 2 - Delaunay triangulation of ``V``
    if isempty(DT)
        DT = AlphaShapes.delaunayTriangulation(V)
    end

    n_upsimplex = length(DT)

    # 3 - process all upper simplex
    ind = 1
    for upper_simplex in DT
        if ind % 500000 == 0
            println(ind," simplices processed of ", n_upsimplex)
        end
        AlphaShapes.processuppersimplex(V,upper_simplex,filtration; digits = digits)
        ind = ind + 1
    end

    return filtration
end


LoadError: UndefVarError: Lar not defined

Nuova versione della funzione alphaFilter

In [2]:
@timeit to "alphaFilter" function alphaFilter(
        V::Lar.Points,
        DT = Array{Int64,1}[];
        digits=64
    )::DataStructures.SortedDict{}

    dim = size(V, 1)
    filtration = DataStructures.SortedDict{Array{Int64,1},Float64}()

    # 1 - Each point => alpha_char = 0.
    @inbounds @simd for i = 1 : size(V, 2)
        insert!(filtration, [i], 0.)
    end

    # 2 - Delaunay triangulation of ``V``
    if isempty(DT)
        DT =Threads.@spawn AlphaShapes.delaunayTriangulation(V)
        DT=fetch(DT)
    end

    n_upsimplex = length(DT)

    # 3 - process all upper simplex
    ind = 1
    @simd for upper_simplex in DT
        if ind % 500000 == 0
            println(ind," simplices processed of ", n_upsimplex)
        end
        AlphaShapes.processuppersimplex(V,upper_simplex,filtration; digits = digits)
        ind = ind + 1
    end

    return filtration
end


LoadError: LoadError: UndefVarError: @timeit not defined
in expression starting at In[2]:1

Vecchia versione della funzione processuppersimplex

In [3]:
function processuppersimplex(
        V::Lar.Points,
        up_simplex::Array{Int64,1},
        filtration::DataStructures.SortedDict{};
        digits=64
        )

    α_char = AlphaShapes.findRadius(V[:, up_simplex], digits=digits);
    insert!(filtration, up_simplex, α_char)

    d = length(up_simplex)-1
    if d > 1
        # It gives back combinations in natural order
        newsimplex = collect(Combinatorics.combinations(up_simplex,d))
        for lowsimplex in newsimplex
            AlphaShapes.processlowsimplex(V, up_simplex, lowsimplex, filtration; digits=digits)
        end
    end
end

LoadError: UndefVarError: Lar not defined

Nuova versione della funzione processuppersimplex

In [4]:
@timeit to "processuppersimplex" function processuppersimplex(
        V::Lar.Points,
        up_simplex::Array{Int64,1},
        filtration::DataStructures.SortedDict{};
        digits=64)


    α_char = AlphaShapes.findRadius(V[:, up_simplex], digits=digits);
    insert!(filtration, up_simplex, α_char)

    d = length(up_simplex)-1
    if d > 1
        # It gives back combinations in natural order
        newsimplex = collect(Combinatorics.combinations(up_simplex,d))
        @simd for lowsimplex in newsimplex
            AlphaShapes.processlowsimplex(V, up_simplex, lowsimplex, filtration; digits=digits)
        end
    end
end

LoadError: LoadError: UndefVarError: @timeit not defined
in expression starting at In[4]:1

Vecchia versione della funzione processlowsimplex

In [5]:
function processlowsimplex(
    V::Lar.Points,
    up_simplex::Array{Int64,1},
    lowsimplex::Array{Int64,1},
    filtration::DataStructures.SortedDict{};
    digits=64)

    α_char = AlphaShapes.findRadius(V[:, lowsimplex], digits=digits)
    point = V[:, setdiff(up_simplex, lowsimplex)]

    if AlphaShapes.vertexInCircumball(V[:, lowsimplex], α_char, point)
        filtration[lowsimplex] = filtration[up_simplex]

    elseif !haskey(filtration, lowsimplex)
        filtration[lowsimplex] = α_char

    end

    d = length(lowsimplex)-1
    if d > 1
        # It gives back combinations in natural order
        newsimplex = collect(Combinatorics.combinations(lowsimplex,d))
        for simplex in newsimplex
             AlphaShapes.processlowsimplex(V, lowsimplex, simplex, filtration, digits=digits)
        end
    end
end

LoadError: UndefVarError: Lar not defined

Nuova versione della funzione processlowsimplex

In [6]:
@timeit to "processlowsimplex" function processlowsimplex(
    V::Lar.Points,
    up_simplex::Array{Int64,1},
    lowsimplex::Array{Int64,1},
    filtration::DataStructures.SortedDict{};
    digits=64)

    α_char = AlphaShapes.findRadius(V[:, lowsimplex], digits=digits)
    point = V[:, setdiff(up_simplex, lowsimplex)]

    if AlphaShapes.vertexInCircumball(V[:, lowsimplex], α_char, point)
        filtration[lowsimplex] = filtration[up_simplex]

    elseif !haskey(filtration, lowsimplex)
        filtration[lowsimplex] = α_char

    end

    d = length(lowsimplex)-1
    if d > 1
        # It gives back combinations in natural order
        newsimplex = collect(Combinatorics.combinations(lowsimplex,d))
        @simd for simplex in newsimplex
             AlphaShapes.processlowsimplex(V, lowsimplex, simplex, filtration, digits=digits)
        end
    end
end

LoadError: LoadError: UndefVarError: @timeit not defined
in expression starting at In[6]:1